In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-12-04...

import pandas as pd
import time
import sqlite3
import random

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of_7 import properties_of
from my_lib.my_db_functions import df_to_sqlite

In [3]:
# Nustatymai, parametrai:

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'
item_params = 'f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&'

table_version = '_7'
item_producer = 'samsung'

common_prop = {'producer':'samsung','5g':True}

url = f'https://www.varle.lt/{item_type}/?{item_params}'
print(url)

https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&


In [4]:
# Šis fragmentas automatiškai išvalo 'items_df' sukauptus duomenis,
# Pasirengta naujų duomenų įrašymui.

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents','producer','5g'))
# čia bus šios programos rezultatas

In [ ]:
# Atidaro Chrome naršyklę,
# reikia patvirtinti "Leisti visus slapukus"!
# Šis fragmentas automatiškai panaikina „draiver“.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

# url = f'https://www.varle.lt/{item_type}/?{item_params}' # iškeliamas į pradžią?

driver.get(url)
# time.sleep(3)
source = driver.page_source
print(f'First page: {url}')

bs = BeautifulSoup(source,'html.parser')
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

In [ ]:
# Toliau analizuojamas prekių sąrašo html iš bs=BeautifulSoup().

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

print(f'DF items: {len(items_df)}.')

In [ ]:
# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

import random

# for page_number in range(2,total_pages+1):
for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?{item_params}p={page_number}'
    driver.get(url)
    time.sleep(random.randint(25,75))
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='last',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# driver.close() # Uždaro Chrome puslapį.

In [ ]:
# Pridedamas kainos stulpelis centais (int)

import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)

items_df['producer'] = 'samsung'
items_df['5g'] = True

items_df

In [ ]:
f'{item_type}{table_version}'

In [ ]:
# Save to SQLite3
df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{table_version}')

In [ ]:
mobilieji_telefonai = properties_of("mobilieji_telefonai",engine="sqlite_file",db_path="./my_scrap_prop_of.db")
mobilieji_telefonai.import_from_df(items_df)

In [ ]:
print(mobilieji_telefonai.export_to_df())
# print(mobilieji_telefonai.export_to_df(['5g','price']))